In [1]:
!pip install optimum[exporters]
!pip install optimum[onnxruntime-gpu]
!apt-get install tensorrt
!mkdir tmp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
from optimum.onnxruntime import ORTModelForQuestionAnswering, ORTModelForFeatureExtraction

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
# model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True).to(device)

tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")

# model = ORTModelForFeatureExtraction.from_pretrained("luotuo-bert-onnx")

provider_options = {

    "trt_engine_cache_enable": True,

    "trt_engine_cache_path": "tmp/trt_cache_luotuo_example"

}

ort_model = ORTModelForFeatureExtraction.from_pretrained("silk-road/luotuo-bert",
                                                         export=True,
                                                         provider="TensorrtExecutionProvider",
                                                         provider_options=provider_options,
                                                         )


inputs = tokenizer("我是谁?", "Using luotuo with ONNX Runtime!", return_tensors="pt")

outputs = ort_model(**inputs)

print(outputs)

assert ort_model.providers == ["TensorrtExecutionProvider", "CUDAExecutionProvider", "CPUExecutionProvider"]




Framework not specified. Using pt to export to ONNX.
Some weights of the model checkpoint at silk-road/luotuo-bert were not used when initializing BertModel: ['mlp.fc.bias', 'mlp.fc.weight', 'mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at silk-road/luotuo-bert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using framework PyTorch: 2.

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

BaseModelOutput(last_hidden_state=tensor([[[-0.0357, -0.0044,  0.0364,  ..., -0.0586, -0.0512, -0.0331],
         [-0.0491, -0.0073,  0.1938,  ..., -0.0451, -0.3034,  0.1354],
         [-0.0247, -0.2675, -0.1023,  ..., -0.0344, -0.3823,  0.1558],
         ...,
         [ 0.0284,  0.0383, -0.1177,  ..., -0.1113, -0.1828,  0.1152],
         [ 0.0918, -0.1546,  0.1217,  ..., -0.1437, -0.2900, -0.0599],
         [-0.0357, -0.0044,  0.0364,  ..., -0.0586, -0.0512, -0.0331]]],
       device='cuda:0'), hidden_states=None, attentions=None)
